In [1]:
import anthropic


In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [3]:
client = anthropic.Anthropic()

In [4]:

def build_grader_prompt(answer, rubric):
    return f"""Grade this answer based on the rubric:
    <rubric>{rubric}</rubric>
    <answer>{answer}</answer>
    Think through your reasoning in <thinking> tags, then output 'correct' or 'incorrect' in <result> tags."""



In [5]:
def grade_completion(output, golden_answer):
    prompt = build_grader_prompt(output, golden_answer)
    print(prompt)
    grader_response = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=2048,
        messages=[{"role": "user", "content": prompt}]
    ).content[0].text
    print("#"*20)
    print(grader_response)
    print("#"*20)

    return "correct" if "correct" in grader_response.lower() else "incorrect"

In [6]:
# Example usage
eval_data = [
    {"question": "Is 42 the answer to life, the universe, and everything?", "golden_answer": "Yes, according to 'The Hitchhiker's Guide to the Galaxy'."},
    {"question": "What is the capital of France?", "golden_answer": "The capital of France is Paris."}
]



In [7]:

def get_completion(prompt: str):
    message = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1024,
        messages=[
        {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text


In [8]:
outputs = [get_completion(q["question"]) for q in eval_data]


In [9]:
outputs

['In Douglas Adams\' science fiction series "The Hitchhiker\'s Guide to the Galaxy," 42 is indeed presented as "the Answer to the Ultimate Question of Life, the Universe, and Everything," as calculated by a supercomputer named Deep Thought over 7.5 million years. Humorously, the characters then realize they don\'t know what the actual question is.\n\nThis has become a popular cultural reference and joke, but it\'s a fictional concept rather than a serious philosophical or scientific answer. The humor lies in the absurdity of reducing existence\'s complexity to a simple number.',
 "The capital of France is Paris. It's not only the capital but also the largest city in France, known for its iconic landmarks like the Eiffel Tower, the Louvre Museum, Notre-Dame Cathedral, and its rich history and culture."]

In [13]:
outputs

['In Douglas Adams\' comedic science fiction series "The Hitchhiker\'s Guide to the Galaxy," 42 is indeed presented as "the Answer to the Ultimate Question of Life, the Universe, and Everything." This answer was calculated by a supercomputer named Deep Thought over 7.5 million years, though humorously, no one knew what the actual question was.\n\nThis reference has become a popular cultural joke and Easter egg in many contexts, but it\'s a fictional concept rather than a serious philosophical or scientific proposition.',
 "The capital of France is Paris. Paris is not only the country's capital but also its largest city and a major cultural, economic, and political center in Europe."]

In [26]:
grades = [grade_completion(output, a["golden_answer"]) for output, a in zip(outputs, eval_data)]


Grade this answer based on the rubric:
    <rubric>Yes, according to 'The Hitchhiker's Guide to the Galaxy'.</rubric>
    <answer>In Douglas Adams' comedic science fiction series "The Hitchhiker's Guide to the Galaxy," 42 is indeed presented as "the Answer to the Ultimate Question of Life, the Universe, and Everything." This answer was calculated by a supercomputer named Deep Thought over 7.5 million years, though humorously, no one knew what the actual question was.

This reference has become a popular cultural joke and Easter egg in many contexts, but it's a fictional concept rather than a serious philosophical or scientific proposition.</answer>
    Think through your reasoning in <thinking> tags, then output 'correct' or 'incorrect' in <result> tags.
####################
<thinking>
The answer correctly identifies that 42 is presented as the "Answer to the Ultimate Question of Life, the Universe, and Everything" in Douglas Adams' "The Hitchhiker's Guide to the Galaxy" series. It pro

In [19]:

print(f"Score: {grades.count('correct') / len(grades) * 100}%")

Score: 100.0%


In [20]:
grades

['correct', 'correct']

In [ ]:
You are a condensed matter physicist to grade an answer in image. 
<rubric>
The following image should correspond to momentum space mesh in the first Brillouin zone of a 2D triangular lattice.
You need to check the point group symmetry.
</rubric>

Think through your reasoning in <thinking> tags, then output 'correct' or 'incorrect' in <result> tags.

In [23]:
def get_gt(paper,directory='../', ):    
    with open(directory+'ground_truth.yaml', 'r') as f:
        gt = yaml.load(f, Loader=yaml.FullLoader)
    for val in gt:
        if val['arxiv'] == float(paper):
            hamiltonian = val['gt']
            symmetry = val['symmetry']
            break
    return val

In [24]:
import yaml

In [25]:
get_gt('2004.04168')

{'arxiv': 2004.04168,
 'gt': "$E_s(k)=\\sum_{n} t_s(n) e^{-i k \\cdot n}$\n$\\hat{H}_{Kinetic} = \\sum_{s, k} E_s(k) c^\\dagger_s(k)  c_s(k)$, where $E_s(k)=\\sum_{n} t_s(n) e^{-i k \\cdot n}$\n$H_{\\text{Hartree}} = \\frac{1}{N} \\sum_{s, s'} \\sum_{k_1, k_2} U(0) \\langle c_s^\\dagger(k_1) c_s(k_1) \\rangle c_{s'}^\\dagger(k_2) c_{s'}(k_2)$\n$H_{\\text{Fock}} = -\\frac{1}{N} \\sum_{s, s'} \\sum_{k_1, k_2} U(k_1 - k_2) \\langle c_s^\\dagger(k_1) c_{s'}(k_1) \\rangle c_{s'}^\\dagger(k_2) c_s(k_2)$\n$U(k) = \\sum_{n} U(n) e^{-i k \\cdot n}$\n",
 'symmetry': 'triangular'}

In [ ]:
d

In [66]:
def generate_evalution_prompt(rubric, image, paper,prompt_template='evaluation_prompt.md', **kwargs):
    
    with open(prompt_template,'r') as f:
        template = f.read()
    with open(rubric,'r') as f:
        rubric = f.read()
    with open(image,'r') as f:
        image = f.read()
    val = get_gt(paper)
    hamiltonian = f"HAMILTONIAN EQUATION: \n{val['gt']} \nLATTICE: {val['symmetry']}"
    return template.format(rubric=rubric.format(lattice=val['symmetry'],**kwargs), hamiltonian=hamiltonian, image_description=image)


In [67]:
print(generate_evalution_prompt(rubric='rubrics1.md', image='image1.md', paper='2004.04168'))

You are a condensed matter physicist working on the numerical calculation for the Hamiltonian of a system using the Hartree Fock method.
You need to evaluate the answer to the <problem> in an image. 
I will provide you with the physical Hamiltonian written in second quantized form as an equation in LaTeX.
You should make your decision based on the following <rubric>:

<rubric>
This image should show momentum space mesh in the first Brillouin zone of the concerned problem.
You need to check the point group symmetry.
</rubric>

The problem is the following:
<problem>
HAMILTONIAN EQUATION: 
$E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$\hat{H}_{Kinetic} = \sum_{s, k} E_s(k) c^\dagger_s(k)  c_s(k)$, where $E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$H_{\text{Hartree}} = \frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(0) \langle c_s^\dagger(k_1) c_s(k_1) \rangle c_{s'}^\dagger(k_2) c_{s'}(k_2)$
$H_{\text{Fock}} = -\frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(k_1 - k_2) \langle c_s^\dagger(k_1) c_{s'}(k

In [68]:
print(generate_evalution_prompt(rubric='rubrics2.md', image='image2.md', paper='2004.04168'))

You are a condensed matter physicist working on the numerical calculation for the Hamiltonian of a system using the Hartree Fock method.
You need to evaluate the answer to the <problem> in an image. 
I will provide you with the physical Hamiltonian written in second quantized form as an equation in LaTeX.
You should make your decision based on the following <rubric>:

<rubric>
This image should show the false color plot with contour lines for non-interacting band structure in the first Brillouin zone for all bands in the problem of interest.
You need to analyze the band structure plots with particular attention to symmetry:

1. First, identify the shape of the Brillouin zone boundary and confirm it matches what's expected for a triangular lattice.
2. Next, examine the symmetry of the energy distribution shown by the false color plot:

    2.1 Identify the highest energy regions (yellow)
    2.2 Trace the pattern of energy variation across the zone
    2.3 Determine what rotational symm

In [69]:
print(generate_evalution_prompt(rubric='rubrics3.md', image='image3.md', paper='2004.04168'))

You are a condensed matter physicist working on the numerical calculation for the Hamiltonian of a system using the Hartree Fock method.
You need to evaluate the answer to the <problem> in an image. 
I will provide you with the physical Hamiltonian written in second quantized form as an equation in LaTeX.
You should make your decision based on the following <rubric>:

<rubric>
This image should show the false color plot for interacting-renormalized band structure in the first Brillouin zone for all bands in the problem of interest, after the self-consistent Hartree-Fock calculation, with the infinitesimal interaction limit.
You need to analyze the band structure plots with particular attention to symmetry and band gap:

1. First, identify the shape of the Brillouin zone boundary and confirm it matches what's expected for a triangular lattice.
2. Next, examine the symmetry of the energy distribution shown by the false color plot:

    2.1 Identify the highest energy regions (yellow)
   

In [72]:
print(generate_evalution_prompt(rubric='rubrics4.md', image='image4.md', paper='2004.04168',U_int=100000))

You are a condensed matter physicist working on the numerical calculation for the Hamiltonian of a system using the Hartree Fock method.
You need to evaluate the answer to the <problem> in an image. 
I will provide you with the physical Hamiltonian written in second quantized form as an equation in LaTeX.
You should make your decision based on the following <rubric>:

<rubric>
This image should show the false color plot for interacting-renormalized band structure in the first Brillouin zone for all bands in the problem of interest, after the self-consistent Hartree-Fock calculation, with the large interaction limit.
You need to analyze the band structure plots with particular attention to symmetry and band gap:

1. First, identify the shape of the Brillouin zone boundary and confirm it matches what's expected for a triangular lattice.
2. Next, examine the symmetry of the energy distribution shown by the false color plot:

    2.1 Identify the highest energy regions (yellow)
    2.2 Tra

In [73]:
response = """Looking at the provided image of band structure calculations from a Hartree-Fock calculation, I'll analyze it according to the rubric:

Brillouin zone boundary shape: ✓ CORRECT
The plots clearly show hexagonal boundaries, which is indeed the correct shape for the first Brillouin zone of a triangular lattice.
Symmetry of the energy distribution:
2.1 Highest energy regions: ✓ CORRECT
The highest energy regions (yellow) are at the center of the Brillouin zone (Γ point) for both bands.
2.2 Energy variation pattern: ✓ CORRECT
The energy decreases radially outward from the center in concentric patterns, showing a smooth gradient from yellow (center) to purple (edges).
2.3 Rotational symmetries: ✓ CORRECT
The energy distribution shows circular/radial symmetry, which includes the 6-fold rotational symmetry expected for a triangular lattice, plus more.
2.4 Symmetry relationship: ✓ CORRECT
Yes, the energy pattern appears to have higher symmetry (nearly continuous rotational symmetry) than the zone boundary (hexagonal D6h symmetry), making the zone boundary symmetry a subgroup of the energy pattern symmetry.
Comparison with isotropic triangular lattice: ✓ CORRECT
The observed energy distribution shows at least D6h symmetry (expected for an isotropic triangular lattice) and actually displays nearly continuous rotational symmetry, which is higher than the minimum required.
Full vs. reduced symmetry determination: ✓ CORRECT
The band structure displays the full symmetry of the triangular lattice, and possibly even higher symmetry with its nearly perfect radial distribution.
Band gap evaluation: ✗ INCORRECT
With U=100000, we would expect a band gap approximately equal to this interaction strength. However, both bands show very similar energy ranges (between approximately -50451 and -50443), with only about 8 energy units difference. This is nowhere near the expected gap of U=100000.

<result>incorrect</result>
The primary issue is with the band gap evaluation. While the symmetry analysis shows the expected properties for a triangular lattice, the band gap between the two bands is significantly smaller than what should result from the very large interaction strength specified in the problem."""

In [76]:
import re

In [79]:
def extract_result_content(string):
    result_tags = re.findall(r"<result>(.*?)</result>", string, re.DOTALL)
    if result_tags:
        result_tags = result_tags[0].strip()
    return result_tags

In [80]:
extract_result_content(response)

'incorrect'

# Task 1

In [ ]:
You are a condensed matter physicist working on the numerical calculation for the Hamiltonian of a system using the Hartree Fock method.
You need to evaluate the answer to the <problem> in an image. 
I will provide you with the physical Hamitonian written in second quantized form as an equation in LaTeX.
You should make your decision based on the following <rubric>:

<rubric>
This image should show momentum space mesh in the first Brillouin zone of the concerned problem.
You need to check the point group symmetry.
</rubric>

The problem is the following:
<problem>
HAMILTONIAN EQUATION: 
$E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$\hat{H}_{Kinetic} = \sum_{s, k} E_s(k) c^\dagger_s(k)  c_s(k)$, where $E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$H_{\text{Hartree}} = \frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(0) \langle c_s^\dagger(k_1) c_s(k_1) \rangle c_{s'}^\dagger(k_2) c_{s'}(k_2)$
$H_{\text{Fock}} = -\frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(k_1 - k_2) \langle c_s^\dagger(k_1) c_{s'}(k_1) \rangle c_{s'}^\dagger(k_2) c_s(k_2)$
$U(k) = \sum_{n} U(n) e^{-i k \cdot n}$

LATTICE: triangular
</problem>

Think through your reasoning in <thinking> tags, then output 'correct' or 'incorrect' in <result> tags. If you are not sure, output 'uncertain' in <result> tags.

# Task 2 (obsolete)

In [27]:
You are a condensed matter physicist to grade an answer in an image. 
<rubric>
The following image should show the false color plot for non-interacting band strcture in the first Brillouin zone for all bands in the problem of interest.
You need to check the following:
1. Point group symmetry of the band structure, including all the rotation and reflection symmetries this system has. This applies to 
1.1: the shape of the first Brillouin zone;
1.2: the false color showing the band structure.
2. The energy degeneracy for different bands. 
</rubric>

The problem is the following:
<problem>
HAMILTONIAN EQUATION: 
$E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$\hat{H}_{Kinetic} = \sum_{s, k} E_s(k) c^\dagger_s(k)  c_s(k)$, where $E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$H_{\text{Hartree}} = \frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(0) \langle c_s^\dagger(k_1) c_s(k_1) \rangle c_{s'}^\dagger(k_2) c_{s'}(k_2)$
$H_{\text{Fock}} = -\frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(k_1 - k_2) \langle c_s^\dagger(k_1) c_{s'}(k_1) \rangle c_{s'}^\dagger(k_2) c_s(k_2)$
$U(k) = \sum_{n} U(n) e^{-i k \cdot n}$

LATTICE: triangular
</problem>

Make sure to check each bullet point in the rubric. Mark them whether they are correct or not.
Finally, print 'correct' or 'incorrect' in <result> tags. 
If you are unsure, print 'uncertain' in <result> tags.

SyntaxError: invalid decimal literal (280044807.py, line 3)

# Task 2

In [ ]:
You are a condensed matter physicist to grade an answer in an image. 
<rubric>
The following image should show the false color plot with contour lines for non-interacting band structure in the first Brillouin zone for all bands in the problem of interest.
You need to analyze the band structure plots with particular attention to symmetry:

1. First, identify the shape of the Brillouin zone boundary and confirm it matches what's expected for a triangular lattice.
2. Next, examine the symmetry of the energy distribution shown by the false color plot:

    2.1 Identify the highest energy regions (yellow)
    2.2 Trace the pattern of energy variation across the zone
    2.3 Determine what rotational symmetries are present in this energy distribution
    2.4 Is the "symmetry of the zone boundary" the subgroup of the "symmetry of the energy pattern"? Namely, the "symmetry of the energy pattern" should contain more symmetries than the "symmetry of the zone boundary".

3. Compare the symmetry you observe in the energy distribution with what would be expected for an isotropic triangular lattice (D6h symmetry).
4. Based on your analysis, determine whether the band structure displays the full symmetry of the triangular lattice or a reduced symmetry.
</rubric>

The problem is the following:
<problem>
HAMILTONIAN EQUATION: 
$E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$\hat{H}_{Kinetic} = \sum_{s, k} E_s(k) c^\dagger_s(k)  c_s(k)$, where $E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$H_{\text{Hartree}} = \frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(0) \langle c_s^\dagger(k_1) c_s(k_1) \rangle c_{s'}^\dagger(k_2) c_{s'}(k_2)$
$H_{\text{Fock}} = -\frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(k_1 - k_2) \langle c_s^\dagger(k_1) c_{s'}(k_1) \rangle c_{s'}^\dagger(k_2) c_s(k_2)$
$U(k) = \sum_{n} U(n) e^{-i k \cdot n}$

LATTICE: triangular
</problem>

Make sure to check each bullet point in the rubric. Mark them whether they are correct or not.
Finally, print 'correct' or 'incorrect' in <result> tags. 
If you are unsure, print 'uncertain' in <result> tags.

# Task 3

In [31]:
You are a condensed matter physicist to grade an answer in an image. 
<rubric>
The following image should show the false color plot false color plot for interacting-renormalized band strcture in the first Brillouin zone for all bands in the problem of interest, after the self-consistent Hartree-Fock calculation, with the infinitesimal interaction limit.
You need to analyze the band structure plots with particular attention to symmetry and band gap:

1. First, identify the shape of the Brillouin zone boundary and confirm it matches what's expected for a triangular lattice.
2. Next, examine the symmetry of the energy distribution shown by the false color plot:

    2.1 Identify the highest energy regions (yellow)
    2.2 Trace the pattern of energy variation across the zone
    2.3 Determine what rotational symmetries are present in this energy distribution
    2.4 Is the "symmetry of the zone boundary" the subgroup of the "symmetry of the energy pattern"? Namely, the "symmetry of the energy pattern" should contain more symmetries than the "symmetry of the zone boundary".

3. Compare the symmetry you observe in the energy distribution with what would be expected for an isotropic triangular lattice (D6h symmetry).
4. Based on your analysis, determine whether the band structure displays the full symmetry of the triangular lattice or a reduced symmetry.
5. Band gap evaluation: Since the interaction is infinitesimal, the band gap should be almost the same as the non-interacting band gap. 
</rubric>

The problem is the following:
<problem>
HAMILTONIAN EQUATION: 
$E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$\hat{H}_{Kinetic} = \sum_{s, k} E_s(k) c^\dagger_s(k)  c_s(k)$, where $E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$H_{\text{Hartree}} = \frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(0) \langle c_s^\dagger(k_1) c_s(k_1) \rangle c_{s'}^\dagger(k_2) c_{s'}(k_2)$
$H_{\text{Fock}} = -\frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(k_1 - k_2) \langle c_s^\dagger(k_1) c_{s'}(k_1) \rangle c_{s'}^\dagger(k_2) c_s(k_2)$
$U(k) = \sum_{n} U(n) e^{-i k \cdot n}$

LATTICE: triangular
</problem>

Make sure to check each bullet point in the rubric. Mark them whether they are correct or not.
Finally, print 'correct' or 'incorrect' in <result> tags. 
If you are unsure, print 'uncertain' in <result> tags.

SyntaxError: invalid syntax (1747439813.py, line 1)

# Task 4

In [ ]:
You are a condensed matter physicist to grade an answer in an image. 
<rubric>
The following image should show the false color plot false color plot for interacting-renormalized band structure in the first Brillouin zone for all bands in the problem of interest, after the self-consistent Hartree-Fock calculation, with the large interaction limit.
You need to analyze the band structure plots with particular attention to symmetry and band gap:

1. First, identify the shape of the Brillouin zone boundary and confirm it matches what's expected for a triangular lattice.
2. Next, examine the symmetry of the energy distribution shown by the false color plot:

    2.1 Identify the highest energy regions (yellow)
    2.2 Trace the pattern of energy variation across the zone
    2.3 Determine what rotational symmetries are present in this energy distribution
    2.4 Is the "symmetry of the zone boundary" the subgroup of the "symmetry of the energy pattern"? Namely, the "symmetry of the energy pattern" should contain more symmetries than the "symmetry of the zone boundary".

3. Compare the symmetry you observe in the energy distribution with what would be expected for an isotropic triangular lattice (D6h symmetry).
4. Based on your analysis, determine whether the band structure displays the full symmetry of the triangular lattice or a reduced symmetry.
5. Band gap evaluation: Since the interaction is very large, the band gap should be almost the same as the interaction strength, U=1e5. 
</rubric>

The problem is the following:
<problem>
HAMILTONIAN EQUATION: 
$E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$\hat{H}_{Kinetic} = \sum_{s, k} E_s(k) c^\dagger_s(k)  c_s(k)$, where $E_s(k)=\sum_{n} t_s(n) e^{-i k \cdot n}$
$H_{\text{Hartree}} = \frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(0) \langle c_s^\dagger(k_1) c_s(k_1) \rangle c_{s'}^\dagger(k_2) c_{s'}(k_2)$
$H_{\text{Fock}} = -\frac{1}{N} \sum_{s, s'} \sum_{k_1, k_2} U(k_1 - k_2) \langle c_s^\dagger(k_1) c_{s'}(k_1) \rangle c_{s'}^\dagger(k_2) c_s(k_2)$
$U(k) = \sum_{n} U(n) e^{-i k \cdot n}$

LATTICE: triangular
</problem>

Make sure to check each bullet point in the rubric. Mark them whether they are correct or not.
Finally, print 'correct' or 'incorrect' in <result> tags. 
If you are unsure, print 'uncertain' in <result> tags.

In [ ]:
Questions:
1. use what form of result as input (numbers or image)
2. use what amoutn of information: Provide the ground truth answer? Provide some symmetry guidance?


In [32]:
Ask for:
1. Show you the results
2. Work on the prompt together to see if any of my knowledge is non standard

SyntaxError: invalid syntax (307638427.py, line 1)